

#Visualización de pipelines de scikit-learn



El objetivo de este notebook es:


1.   Realizar un mini proyecto de clasificación utilizando un pipeline
2.   Visualizar el pipeline
3.   Imputar valores faltantes con la media de los datos


## Primero cargamos el set de datos "AmesHousing.csv"

Definimos los datos y el objetivo. En este caso, construimos un modelo de clasificación en el que se identifiquen las casas que cuestán más de 200,000 dólares y las cuestan menos o igual a 200,000 dólares.

In [1]:
import pandas as pd

ames_housing = pd.read_csv("AmesHousing.csv")
target_name = "SalePrice"
data, target = (
    ames_housing.drop(columns=target_name),
    ames_housing[target_name],
)
target = (target > 200_000).astype(int)

Inspeccionamos las primeras filas del conjunto de datos

In [2]:
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition
0,1,526301100,20,RL,141.0,31770.0,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
1,2,526350040,20,RH,80.0,NaN,Pave,NaN,Reg,Lvl,...,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
2,3,526351010,20,RL,81.0,14267.0,Pave,NaN,IR1,Lvl,...,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
3,4,526353030,20,RL,93.0,11160.0,Pave,NaN,Reg,Lvl,...,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal
4,5,527105010,60,RL,74.0,NaN,Pave,NaN,IR1,Lvl,...,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal


Por simplicidad, vamos a seleccionar solo algunas características.

In [3]:
numeric_features = ["Lot Area", "Full Bath", "Half Bath"]
categorical_features = ["Neighborhood", "House Style"]
data = data[numeric_features + categorical_features]

Inspeccionamos los primeros 15 datos

In [4]:
data.head(15)

,Lot Area,Full Bath,Half Bath,Neighborhood,House Style
0,31770.0,1.0,0.0,NAmes,1Story
1,NaN,1.0,NaN,NAmes,1Story
2,14267.0,NaN,1.0,NAmes,1Story
3,11160.0,2.0,1.0,NAmes,1Story
4,NaN,2.0,NaN,Gilbert,2Story
5,9978.0,2.0,1.0,Gilbert,2Story
6,4920.0,NaN,0.0,StoneBr,1Story
7,5005.0,2.0,0.0,StoneBr,1Story
8,5389.0,2.0,0.0,StoneBr,1Story
9,NaN,NaN,1.0,Gilbert,2Story


Vemos que existen datos faltantes (NaN). Veamos cuántos son:

In [5]:
data.isnull().sum()

,0
Lot Area,3
Full Bath,3
Half Bath,2
Neighborhood,0
House Style,0


Cuando se tienen datos faltantes (NaN) en el dataset, se puede utilizar una técnica de imputación para completar esos datos faltantes en lugar de simplemente borrar las observaciones que tengan dichos datos faltantes. Una forma de imputar los datos faltantes es utilizando la media o la mediana de los datos (por característica). Sin embargo, como la media es muy susceptible a los outliers, se recomienda utilizar la mediana. Veamos un ejemplo de cómo imputar los datos faltantes utilizando `SimpleImputer`.

In [6]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")
imputed_data = pd.DataFrame(imputer.fit_transform(data[numeric_features]),columns=numeric_features)
imputed_data.head(15)

,Lot Area,Full Bath,Half Bath
0,31770.0,1.0,0.0
1,9434.0,1.0,0.0
2,14267.0,2.0,1.0
3,11160.0,2.0,1.0
4,9434.0,2.0,0.0
5,9978.0,2.0,1.0
6,4920.0,2.0,0.0
7,5005.0,2.0,0.0
8,5389.0,2.0,0.0
9,9434.0,2.0,1.0


In [7]:
imputed_data.isnull().sum()

,0
Lot Area,0
Full Bath,0
Half Bath,0


Podemos ver que ya no hay datos faltantes en este ejemplo.

## Ahora creamos el pipeline

Lo primero es definir los pasos del preprocesamiento: aplicaremos un escalamiento estándar y un imputador de datos para las características numéricas. Así como una codificación one-hot para las características categóricas.

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        (
            "scaler",
            StandardScaler(),
        ),
    ]
)

categorical_transformer = OneHotEncoder(handle_unknown="ignore")

El siguiente paso es aplicar las transformaciones usando `ColumnTransformer`

In [9]:
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ]
)

Ahora definimos el modelo con un pipeline que incluya el preprocesamiento y el clasificador de regresión logística.

In [10]:
from sklearn.linear_model import LogisticRegression

model = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression()),
    ]
)

Visualicemos el modelo

In [11]:
model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  ['Lot Area', 'Full Bath',
                                                   'Half Bath']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['Neighborhood',
                                                   'House Style'])])),
                ('classifier', LogisticRegression())])

## Finalmente, entrenemos el modelo y validemos el desempeño utilizando validación cruzada

In [12]:
from sklearn.model_selection import cross_validate

cv_results = cross_validate(model, data, target, cv=5)
scores = cv_results["test_score"]
print(
    "The mean cross-validation accuracy is: "
    f"{scores.mean():.3f} ± {scores.std():.3f}"
)

The mean cross-validation accuracy is: 0.860 ± 0.011


Nota que...

En este caso, alrededor el 86% de las veces el modelo predice correctamente si el precio de una casa está por encima o por debajo del umbral de 200,000 dólares. Pero ten en cuenta que esta puntuación se obtuvo seleccionando algunas características a mano, lo que no es necesariamente lo mejor que podemos hacer para esta tarea de clasificación. En este ejemplo, podemos esperar que ajustar un pipeline de machine learning más complejo con un conjunto de características más rico puede mejorar este nivel de rendimiento.

Además, reducir un problema de estimación de precios a un problema de clasificación binaria con un único umbral de 200,000 dólares es probablemente demasiado impreciso para ser útil en la práctica. Tratar este problema como un problema de regresión probablemente sea una mejor idea. Veremos más adelante cómo entrenar y evaluar el rendimiento de varios modelos de regresión.
